<a href="https://colab.research.google.com/github/KurusuBlueCat/deep-lob/blob/main/colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/

/content


In [ ]:
!npx degit KurusuBlueCat/deep-lob -f

npx: installed 1 in 1.9s
> destination directory is not empty. Using --force, continuing
> cloned KurusuBlueCat/deep-lob#HEAD


In [ ]:
!pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.6/489.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.0/441.0 kB 40.1 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.3.3
    Uninstalling flatbuffers-23.3.3:
      Successfully uninstalled flatbuffers-23.3.3


In [ ]:
import keras
import tensorflow as tf
from keras.utils import Sequence
from keras import layers
from keras.callbacks import CSVLogger
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
import json
from dataclasses import dataclass, asdict
import ast
import os
from typing import Union, List
from datetime import datetime, timedelta
import time

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
import stride_data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#this path should link to a folder in your drive that contains the data required.
#will also output run report there at output/ folder
gdrive_data_folder = "/content/drive/MyDrive/github_repo/deep-lob"

In [ ]:
from collections import namedtuple
Chunk = namedtuple('Chunk', ['lob', 'factor', 'label'])

In [ ]:
def get_factor_itr(factor_file_list=[f'factor_22.csv', 'factor_23.csv'], base_folder=f'{gdrive_data_folder}/factor',
                   chunksize=500000):
    itr_list = []
    for f_name in factor_file_list:
        itr_list.append(pd.read_csv(fr'{base_folder}/{f_name}', chunksize=chunksize))

    for df_tuple in zip(*itr_list):
        df_list = []
        for df in df_tuple:
            df['time'] = pd.to_datetime(df['time'])
            df = df.set_index('time').sort_index()
            # print(df.index.min(), df.index.max())
            df = df[~df.index.duplicated()]
            # print(df.index.min(), df.index.max())
            df_list.append(df)
        yield pd.concat(df_list, axis=1)

In [ ]:
chunksize = 100000

all_feature = ['volume', 'bid_1', 'bid_1', 'bid_size_1', 'ask_1',
       'ask_size_1', 'bid_2', 'bid_size_2', 'ask_2', 'ask_size_2', 'bid_3',
       'bid_size_3', 'ask_3', 'ask_size_3', 'bid_4', 'bid_size_4', 'ask_4',
       'ask_size_4', 'bid_5', 'bid_size_5', 'ask_5', 'ask_size_5', 'vwap']

lob_feature = ['bid_1', 'bid_size_1', 'ask_1',
               'ask_size_1', 'bid_2', 'bid_size_2', 'ask_2', 'ask_size_2', 'bid_3',
               'bid_size_3', 'ask_3', 'ask_size_3', 'bid_4', 'bid_size_4', 'ask_4',
               'ask_size_4', 'bid_5', 'bid_size_5', 'ask_5', 'ask_size_5']

def get_chunk_from_file_gen():
    data_itr = pd.read_csv(f'{gdrive_data_folder}/data/data_night_shifted_au.csv.gz', chunksize=chunksize)
    label_itr = pd.read_csv(f'{gdrive_data_folder}/label/2_min_tp4_sl2_10yuan_1delay_target.csv', chunksize=chunksize)
    factor_itr = get_factor_itr(['factor_24.csv', 'factor_23.csv'], chunksize=chunksize)

    for df, fac, label in zip(data_itr, factor_itr, label_itr):
        df['time'] = pd.to_datetime(df['time'])
        df = df.set_index('time').sort_index()

        label['time'] = pd.to_datetime(label['time'])
        label = label.set_index('time').sort_index()

        df = df.loc[~df.index.duplicated()]
        fac = fac.loc[~fac.index.duplicated()]
        label = label.loc[~label.index.duplicated()]
        # print(df.index.max(), label.index.max(), fac.index.max())
        if (fac.index.max() != label.index.max()) or (fac.index.max() != df.index.max()):
            raise RuntimeError("Index do not match!")

        yield Chunk(df, fac, label), label.index.max()

def concat_chunk(*chunk_tuple):
    arg_list = []
    for df_tuple in zip(*chunk_tuple):
        df = pd.concat(df_tuple).sort_index()
        df = df.loc[~df.index.duplicated()]
        arg_list.append(df)
    return Chunk(*arg_list)

def get_weekly_data_gen(skip_to=None):
    print('Data loading started')
    chunk_gen = get_chunk_from_file_gen()
    while True:
        chunk, latest = next(chunk_gen)
        print(f"                                         ", end='\r')
        time.sleep(0)
        print(f"{latest=}", end='\r')
        time.sleep(0)
        if skip_to is not None:
            if latest < skip_to:
                continue

        if (chunk.label.index.day_of_week == 0).sum() > 0:
            start = chunk.label[chunk.label.index.day_of_week == 0].index.date.min()
            start: pd.Timestamp = pd.Timestamp(start)
            end = start + pd.Timedelta(5,'d') #end on friday midnight
            break

    chunk = Chunk(*[data[data.index >= start] for data in chunk])
    for new_chunk, latest in chunk_gen:
        chunk = concat_chunk(chunk, new_chunk)
        while latest > end:
            print(f"                                                  ", end='\r')
            time.sleep(0)
            print(f"{latest=}", end='\r')
            time.sleep(0)
            to_yield = Chunk(*(df.loc[start:end] for df in chunk))
            chunk = Chunk(*(df.loc[end:] for df in chunk))
            start = start + pd.Timedelta(7,'d')
            end = end + pd.Timedelta(7,'d')
            yield to_yield

def get_chunk_list_gen(list_len=2, skip_to=None, end=None):
    out_list = []
    weekly_gen = get_weekly_data_gen(skip_to)
    for week_chunk in weekly_gen:
        if (end is not None) and (week_chunk.label.index.max() > end):
            break 
        out_list.append(week_chunk)
        if len(out_list) < list_len:
            continue
        if len(out_list) > list_len:
            out_list = out_list[1:] #basically, pop left
        yield out_list

def split_market(chunk, avoid_market_edge = timedelta(minutes=5)):
    chunk_arg = []
    for df in chunk:
        night_df = df.loc[(df.index.time >= (datetime(1970,1,1,21,0,0)+avoid_market_edge).time()) 
                        | (df.index.time <= (datetime(1970,1,1,2,30,0)-avoid_market_edge).time())] #night market
        morning_df = df.loc[(df.index.time >= (datetime(1970,1,1,9,0,0)+avoid_market_edge).time()) 
                            & (df.index.time <= (datetime(1970,1,1,11,30,0)-avoid_market_edge).time())] #morning market
        afternoon_df = df.loc[(df.index.time >= (datetime(1970,1,1,13,30,0)+avoid_market_edge).time())
                            & (df.index.time <= (datetime(1970,1,1,15,0,0)-avoid_market_edge).time())] #afternoon market
        chunk_arg.append((morning_df, afternoon_df, night_df))

    return tuple(Chunk(*df_list) for df_list in zip(*chunk_arg))

In [ ]:

# training_week = 8
# skip_to = pd.Timestamp('2022-11-01 00:00:00.000') #use this to skip to the period we want
# # skip_to = None
# # end = pd.Timestamp('2022-07-15 23:59:59.500') #use this to fix when to stop training
# end = pd.Timestamp('2023-01-01 00:00:00')

# chunk_list_gen = get_chunk_list_gen(training_week+1, skip_to=skip_to, end=end)
# for i, chunk_list in enumerate(chunk_list_gen):
#     break

# train_chunk = concat_chunk(*chunk_list[:training_week])
# walk_forward_chunk = chunk_list[-1]

# morning_train_chunk, afternoon_train_chunk, night_train_chunk = split_market(train_chunk)
# morning_wf_chunk, afternoon_wf_chunk, night_wf_chunk = split_market(walk_forward_chunk)

In [ ]:
# lookback = 50 #this number of timestep per batch
# batch_size = 1000 #this number of samples in a batch

# chunk = morning_train_chunk

# sp_list_list = []
# for chunk in [morning_train_chunk, afternoon_train_chunk, night_train_chunk]:

#     X = chunk.lob.loc[:, all_feature].join(chunk.factor)
#     y = chunk.label

#     sp_list : List[stride_data.SequencePair] = stride_data.create_train_val_sequence_cv(X, y.iloc[:, 0], cv=4, lookback=lookback, 
#                                             batch_size=batch_size, batch_no=None, 
#                                             shuffle=False)

#     sp_list_list.append(sp_list)

# sequence_pair_list_per_cv = list(zip(*sp_list_list))
# sequence_list_per_cv = [stride_data.CombinedSequence(*[sequence_pair_list_per_cv[i][j].train_sequence 
#                         for j in range(len(sequence_pair_list_per_cv[i]))])
#                         for i in range(len(sequence_pair_list_per_cv))]

In [ ]:
def make_model_original(lookback, feature_no, lob_unit, inception_unit):

    input_layer = layers.Input(shape=(lookback, feature_no))
    lob_layer = layers.Reshape(target_shape=(lookback,feature_no,1))(input_layer)

    cnn_act = 'leaky_relu'

    lob_layer = layers.Conv2D(filters=lob_unit, kernel_size=(1,2), strides=(1,2), activation=cnn_act)(lob_layer)
    lob_layer = layers.Conv2D(filters=lob_unit, kernel_size=(5,1), padding='same', activation=cnn_act)(lob_layer)
    lob_layer = layers.Conv2D(filters=lob_unit, kernel_size=(1,2), strides=(1,2), activation=cnn_act)(lob_layer)
    lob_layer = layers.Conv2D(filters=lob_unit, kernel_size=(5,1), padding='same', activation=cnn_act)(lob_layer)
    lob_layer = layers.Conv2D(filters=lob_unit, kernel_size=(1,5), activation=cnn_act)(lob_layer)
    lob_layer = layers.Conv2D(filters=lob_unit, kernel_size=(5,1), padding='same', activation=cnn_act)(lob_layer)

    inception_act = 'leaky_relu'

    inception_1 = layers.Conv2D(inception_unit, (1,1), padding='same', activation=inception_act)(lob_layer)
    inception_1 = layers.Conv2D(inception_unit, (3,1), padding='same', activation=inception_act)(inception_1)

    inception_2 = layers.Conv2D(inception_unit, (1,1), padding='same', activation=inception_act)(lob_layer)
    inception_2 = layers.Conv2D(inception_unit, (5,1), padding='same', activation=inception_act)(inception_2)

    inception_3 = layers.MaxPooling2D((3,1), strides=(1,1), padding='same')(lob_layer)
    inception_3 = layers.Conv2D(inception_unit, (1,1), padding='same', activation=inception_act)(inception_3)

    inception = tf.keras.layers.concatenate([inception_1, inception_2, inception_3], axis=3)
    inception = layers.Reshape((lookback, inception_unit*3))(inception)

    lstm = layers.LSTM(32)(inception)
    output = layers.Dense(3, activation='softmax')(lstm)

    model = keras.Model(input_layer, output)

    return model

def make_model_mz_prelu(lookback, feature_no, lob_unit, inception_unit):

    input_layer = layers.Input(shape=(lookback, feature_no))
    lob_layer = layers.Reshape(target_shape=(lookback,feature_no,1))(input_layer)

    cnn_act = 'linear'

    lob_layer = layers.Conv2D(filters=lob_unit, kernel_size=(1,2), strides=(1,2), activation=cnn_act)(lob_layer)
    lob_layer = layers.PReLU()(lob_layer)
    lob_layer = layers.Conv2D(filters=lob_unit, kernel_size=(5,1), padding='same', activation=cnn_act)(lob_layer)
    lob_layer = layers.PReLU()(lob_layer)
    lob_layer = layers.Conv2D(filters=lob_unit, kernel_size=(1,2), strides=(1,2), activation=cnn_act)(lob_layer)
    lob_layer = layers.PReLU()(lob_layer)
    lob_layer = layers.Conv2D(filters=lob_unit, kernel_size=(5,1), padding='same', activation=cnn_act)(lob_layer)
    lob_layer = layers.PReLU()(lob_layer)
    lob_layer = layers.Conv2D(filters=lob_unit, kernel_size=(1,5), activation=cnn_act)(lob_layer)
    lob_layer = layers.PReLU()(lob_layer)
    lob_layer = layers.Conv2D(filters=lob_unit, kernel_size=(5,1), padding='same', activation=cnn_act)(lob_layer)
    lob_layer = layers.PReLU()(lob_layer)

    inception_act = 'linear'

    inception_1 = layers.Conv2D(inception_unit, (1,1), padding='same', activation=inception_act)(lob_layer)
    inception_1 = layers.PReLU()(inception_1)
    inception_1 = layers.Conv2D(inception_unit, (3,1), padding='same', activation=inception_act)(inception_1)
    inception_1 = layers.PReLU()(inception_1)

    inception_2 = layers.Conv2D(inception_unit, (1,1), padding='same', activation=inception_act)(lob_layer)
    inception_2 = layers.PReLU()(inception_2)
    inception_2 = layers.Conv2D(inception_unit, (5,1), padding='same', activation=inception_act)(inception_2)
    inception_2 = layers.PReLU()(inception_2)

    inception_3 = layers.MaxPooling2D((3,1), strides=(1,1), padding='same')(lob_layer)
    inception_3 = layers.PReLU()(inception_3)
    inception_3 = layers.Conv2D(inception_unit, (1,1), padding='same', activation=inception_act)(inception_3)
    inception_3 = layers.PReLU()(inception_3)

    inception = tf.keras.layers.concatenate([inception_1, inception_2, inception_3], axis=3)
    inception = layers.Reshape((lookback, inception_unit*3))(inception)

    lstm = layers.LSTM(32)(inception)
    output = layers.Dense(3, activation='softmax')(lstm)

    model = keras.Model(input_layer, output)

    return model

model_maker_dict = {
    'original' : make_model_original,
    'mz_prelu' : make_model_mz_prelu,
}

In [ ]:
schematic_dict = {
    'stride2_depth_simple': [
        (layers.Reshape, {'target_shape': (100,-1,1)}), #50 lookback
        (layers.Conv2D, {'filters': 16, 'kernel_size': (1, 2), 'strides': (1, 2)}),
        (layers.BatchNormalization, {}),
        (layers.Conv2D, {'filters': 16, 'kernel_size': (5, 1), 'padding': 'same', 'activation': 'leaky_relu'}),
        (layers.BatchNormalization, {}),
        (layers.Conv2D, {'filters': 16, 'kernel_size': (1, 2), 'strides': (1, 2)}),
        (layers.BatchNormalization, {}),
        (layers.Conv2D, {'filters': 16, 'kernel_size': (5, 1), 'padding': 'same', 'activation': 'leaky_relu'}),
        (layers.BatchNormalization, {}),
        (layers.Conv2D, {'filters': 16, 'kernel_size': (1, 5), 'activation': 'leaky_relu'}),
        (layers.BatchNormalization, {}),
        (layers.Conv2D, {'filters': 16, 'kernel_size': (5, 1), 'padding': 'same', 'activation': 'leaky_relu'}),
        (layers.Reshape, {'target_shape': (100,-1)}), #50 lookback
        (layers.BatchNormalization, {}),
        (layers.LSTM, {'units': 32, 'kernel_regularizer': 'l2'}),
        (layers.BatchNormalization, {}),
    ],
}

opt_dict = {
    'Adam': tf.keras.optimizers.Adam,
    'Nadam': tf.keras.optimizers.Nadam,
}

# try

for k, s in schematic_dict.items():
    for l, p in s:
        l(**p)

In [ ]:
from sklearn.model_selection import ParameterGrid

metric_dict = {
    # 'mse': tf.keras.metrics.MeanSquaredError(name='mean_squared_error', dtype=None),
    # 'mae': tf.keras.metrics.MeanAbsoluteError(name='mean_absolute_error', dtype=None),
    'coss': tf.keras.metrics.CosineSimilarity(name='cosine_similarity', dtype=None),
    'ce': tf.keras.metrics.CategoricalCrossentropy(name='categorical_ce', dtype=None)
}

cosine = tf.keras.losses.CosineSimilarity(name='cosine_similarity')
mse = tf.keras.losses.MeanSquaredError(name='mean_squared_error')
mae = tf.keras.losses.MeanAbsoluteError(name='mean_absolute_error')

ce = tf.keras.losses.CategoricalCrossentropy(name='categorical_ce')
fc = tf.keras.losses.CategoricalFocalCrossentropy()

loss_dict = {
    # 'mse': tf.keras.losses.MeanSquaredError(name='mean_squared_error', reduction="auto"),
    # 'mae': tf.keras.losses.MeanAbsoluteError(name='mean_absolute_error', reduction="auto"),
    'coss': tf.keras.losses.CosineSimilarity(name='cosine_similarity', reduction="auto"),
    'ce': tf.keras.losses.CategoricalCrossentropy(name='categorical_ce', reduction='auto'),
    'fc': tf.keras.losses.CategoricalFocalCrossentropy(),
    # 'ce': tf.keras.losses.BinaryFocalCrossentropy(name='categorical_ce', reduction='auto'),
    # 'coss-mse': lambda y, yhat: mse(y, yhat) + cosine(y, yhat),
    # 'coss-mae': lambda y, yhat: mae(y, yhat) + cosine(y, yhat),
}

#           'features' :  [['High', 'Low', 'Open', 'Close', 
#                          'vix_forward_5_historical', 'vix_forward_10_historical', 'vix_forward_15_historical']],

p_grid = {'batch_size' : [128],
          'features' :  [lob_feature],
          'init_learning_rate' : [0.001],
          'lr_decay' : [1.0],
          'loss': ['fc'],
          'schematic' : [
#                          'cnn_1',
                        #  'cnn_2',
#                          'cnn_3',
#                          'tuesday_1',
#                          'tuesday_2', 
                        #  'midnight_1',]}
                         'mz_prelu',]}
#                          'midnight_6_2_rnn',
                        #  'midnight_6_rnn',]}
#                          'midnight_7',
#                          'midnight_8',
#                          'midnight_9',
#                          'friday_1',
#                          'friday_2',
#                          'midnight_6_3',
#                          'midnight_6_4',
#                          'midnight_6_5',]}
#                          'midnight_6_6',]}
#                          'midnight_6_7']}

p_grid = ParameterGrid(p_grid)
len(p_grid)

1

In [ ]:
lob_feature

In [ ]:
def make_if_not_exist(folder_name, base_folder=f'{gdrive_data_folder}/output'):
    try:
        os.makedirs(fr'{base_folder}/{folder_name}')
    except:
        pass

In [ ]:
from dataclasses import field, dataclass

@dataclass
class HyperParameters:
    features: list = field(default_factory=lambda: list(all_feature))
    lookback: int = 50
    epochs: int = 100
    cv: int = 4
    batch_size: int = 128
    batch_no: tuple = (None, None, None) #per market period. Mornin afternoon night.
    shuffle: bool = True
    init_learning_rate: float = 2.5e-1
    seed: int = 420
    lr_decay: float = 0.95
    trim: str = 'both'
    decay_steps: int = 2000
    schematic: str = 'midnight_1'
    opt: str = 'Adam'
    replace: bool = False
    loss: str = 'ce'
    training_week: int = 8
    # skip_to = pd.Timestamp('2022-07-02 11:24:58.000') #use this to skip to the period we want
    skip_to: pd.Timestamp = pd.Timestamp('2022-11-01 00:00:00.000')
    # end = pd.Timestamp('2022-07-15 23:59:59.500') #use this to fix when to stop training
    end: pd.Timestamp = pd.Timestamp('2023-03-01 00:00:00')

hp_list = [HyperParameters(**p) for p in p_grid]
base_output_folder = f'{gdrive_data_folder}/output'

run_prefix = 'test'

model_dict = {}

make_if_not_exist(fr"plots/{run_prefix}", base_output_folder)
make_if_not_exist(fr"output/{run_prefix}", base_output_folder)
make_if_not_exist(fr"callback_logs/{run_prefix}", base_output_folder)
make_if_not_exist(fr"callback_logs/{run_prefix}/_temp", base_output_folder)
make_if_not_exist(fr"run_summary/{run_prefix}", base_output_folder)
    
# if run_prefix not in [d for d in os.listdir('output\\') if os.path.isdir('output\\' + d)]:
#     os.makedirs('output\\' + run_prefix)

In [ ]:
continue_loop = 0
end_loop = 9999

for run_count, hp in enumerate(hp_list[continue_loop:]):
    run_count += continue_loop
    if run_count >= end_loop:
        break
    print(hp)
    if hp.trim == 'both':
        hp.trim = (hp.lookback, hp.lookback)
    break



HyperParameters(features=['bid_1', 'bid_size_1', 'ask_1', 'ask_size_1', 'bid_2', 'bid_size_2', 'ask_2', 'ask_size_2', 'bid_3', 'bid_size_3', 'ask_3', 'ask_size_3', 'bid_4', 'bid_size_4', 'ask_4', 'ask_size_4', 'bid_5', 'bid_size_5', 'ask_5', 'ask_size_5'], lookback=50, epochs=100, cv=4, batch_size=128, batch_no=(None, None, None), shuffle=True, init_learning_rate=0.001, seed=420, lr_decay=1.0, trim='both', decay_steps=2000, schematic='mz_prelu', opt='Adam', replace=False, loss='fc', training_week=8, skip_to=Timestamp('2022-11-01 00:00:00'), end=Timestamp('2023-03-01 00:00:00'))


In [ ]:
chunk_list_gen = get_chunk_list_gen(hp.training_week+1, skip_to=hp.skip_to, end=hp.end)
for i, chunk_list in enumerate(chunk_list_gen):
    break

train_chunk = concat_chunk(*chunk_list[:hp.training_week])
walk_forward_chunk = chunk_list[-1]

morning_train_chunk, afternoon_train_chunk, night_train_chunk = split_market(train_chunk)
morning_wf_chunk, afternoon_wf_chunk, night_wf_chunk = split_market(walk_forward_chunk)

Data loading started


In [ ]:
def make_class(label, epsilon=1.0001):
    long_wins = label['long_wealth'] > label['short_wealth']

    class_label = pd.Series(np.nan, index=label.index)
    class_label[label.notna().all(1)] = 0
    class_label[long_wins & (label['long_wealth'] > epsilon)] = 1
    class_label[~long_wins & (label['short_wealth'] > epsilon)] = -1
    class_label = class_label.dropna().astype(int)
    return pd.get_dummies(class_label, columns=['short', 'neutral', 'long']).astype(int)

make_class(night_train_chunk.label, 1.0000).mean()

-1    0.182673
 0    0.623811
 1    0.193516
dtype: float64

In [ ]:
X_list = []
y_list = []

epsilon=1.0000

for i, chunk in enumerate([morning_train_chunk, afternoon_train_chunk, night_train_chunk]):
    X = chunk.lob.loc[:, hp.features]
    y = make_class(chunk.label, epsilon)
    X = X.reindex(y.index)
    y = y.reindex(X.index)
    X_list.append(X)
    y_list.append(y)

scaler = StandardScaler()
scaler.fit(pd.concat(X_list));

In [ ]:
print(morning_train_chunk.label.dropna().shape[0])
print(afternoon_train_chunk.label.dropna().shape[0])
print(night_train_chunk.label.dropna().shape[0])
print(morning_train_chunk.label.dropna().shape[0] + afternoon_train_chunk.label.dropna().shape[0] + night_train_chunk.label.dropna().shape[0])

228842
132460
519114
880416


In [ ]:
sp_generator_list = []

for i, chunk in enumerate([morning_train_chunk, afternoon_train_chunk, night_train_chunk]):

    X = chunk.lob.loc[:, hp.features]
    X = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)
    y = make_class(chunk.label, epsilon)
    X = X.reindex(y.index)
    y = y.reindex(X.index)
    #transform y here

    sp_generator : List[stride_data.SequencePair] = stride_data.create_train_val_sequence_cv(
        X, y, cv=hp.cv, lookback=hp.lookback, 
        batch_size=hp.batch_size, batch_no=hp.batch_no[i], 
        shuffle=hp.shuffle, trim=hp.trim, replace=hp.replace)

    sp_generator_list.append(sp_generator)


In [ ]:
wf_sequence_list = []
for i, chunk in enumerate([morning_wf_chunk, afternoon_wf_chunk, night_wf_chunk]):

    X = chunk.lob.loc[:, hp.features]
    X = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)
    y = make_class(chunk.label, epsilon)
    X = X.reindex(y.index)
    y = y.reindex(X.index)
    #transform y here

    wf_sequence : stride_data.StrideData = stride_data.StrideData(X, y, lookback=hp.lookback, 
                                            batch_size=y.shape[0] - hp.lookback, batch_no=None, 
                                            shuffle=False, replace=False)

    wf_sequence_list.append(wf_sequence)

In [ ]:
early_stopper = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=15,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

model_list = []

for cv_count in range(hp.cv):
    sp_list = [next(sp_gen) for sp_gen in sp_generator_list]
    sq = stride_data.CombinedSequence(*[sp.train_sequence for sp in sp_list], shuffle=hp.shuffle, seed=hp.seed, replace=hp.replace)
    test_tuple = np.vstack([sp.test_tuple[0] for sp in sp_list]), np.vstack([sp.test_tuple[1] for sp in sp_list])

    print('running cv' + str(cv_count))
    csv_logger = CSVLogger(fr'{base_output_folder}\callback_logs\{run_prefix}\_temp\{run_prefix}_{cv_count}.csv')
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=hp.init_learning_rate,
        decay_steps=hp.decay_steps,
        decay_rate=hp.lr_decay)
    try:
        opt = opt_dict[hp.opt](learning_rate=lr_schedule)
    except Exception:
        print(f"{hp.opt} does not work with scheduler")
        opt = opt_dict[hp.opt](learning_rate=hp.init_learning_rate)

    # model = tf.keras.Sequential()
    # for l, p in schematic_dict[hp.schematic]:
    #     model.add(l(**p))
        
    # model.add(layers.Dense(3, activation='softmax')) # add output node

    model_maker = model_maker_dict[hp.schematic]

    model = model_maker(hp.lookback, len(hp.features), 16, 16)

    model.compile(loss=loss_dict[hp.loss], optimizer=opt, metrics=[], weighted_metrics=[])
    # with tf.device('/cpu:0'):
    model.fit(x=sq,
            use_multiprocessing=False,
            validation_data=test_tuple,
            epochs=hp.epochs,
            verbose=1,
            callbacks=[csv_logger, early_stopper])
    
    model_list.append(model)

running cv0
Epoch 1/100
5153/5153 [==============================] - 109s 18ms/step - loss: 0.1002 - val_loss: 0.0956
Epoch 2/100
5153/5153 [==============================] - 89s 17ms/step - loss: 0.0980 - val_loss: 0.0998
Epoch 3/100
5153/5153 [==============================] - 90s 17ms/step - loss: 0.0938 - val_loss: 0.1019
Epoch 4/100
5153/5153 [==============================] - 89s 17ms/step - loss: 0.0886 - val_loss: 0.1124
Epoch 5/100
5153/5153 [==============================] - 89s 17ms/step - loss: 0.0838 - val_loss: 0.1251
Epoch 6/100
5153/5153 [==============================] - 88s 17ms/step - loss: 0.0788 - val_loss: 0.1298
Epoch 7/100
5153/5153 [==============================] - 89s 17ms/step - loss: 0.0737 - val_loss: 0.1360
Epoch 8/100
5153/5153 [==============================] - 89s 17ms/step - loss: 0.0687 - val_loss: 0.1525
Epoch 9/100
5153/5153 [==============================] - 88s 17ms/step - loss: 0.0636 - val_loss: 0.1637
Epoch 10/100
5153/5153 [==================

KeyboardInterrupt: ignored

In [ ]:
model

output = model.predict(wf_sequence_list[0][0][0])

1094/1094 [==============================] - 5s 4ms/step


In [ ]:
true_df = pd.DataFrame(wf_sequence_list[0][0][1], columns=['true_short','true_neutral','true_long'])

In [ ]:
predicted = pd.DataFrame(output, columns=['short','neutral','long']).join(true_df)
predicted

,short,neutral,long,true_short,true_neutral,true_long
0,0.003413,0.427331,0.569256,0,1,0
1,0.000202,0.303920,0.695878,1,0,0
2,0.000226,0.337788,0.661986,1,0,0
3,0.000563,0.335852,0.663585,1,0,0
4,0.005741,0.506746,0.487513,1,0,0
...,...,...,...,...,...,...
34990,0.226024,0.747509,0.026467,0,0,1
34991,0.338429,0.647026,0.014546,0,0,1
34992,0.466474,0.525126,0.008400,0,0,1
34993,0.463313,0.514097,0.022590,0,1,0


In [ ]:
predicted[predicted['short'] == predicted.iloc[:, :3].max(1)].iloc[:,3:].mean()

true_short      0.170662
true_neutral    0.612461
true_long       0.216877
dtype: float64

In [ ]:
predicted[predicted['neutral'] == predicted.iloc[:, :3].max(1)].iloc[:,3:].mean()

true_short      0.166731
true_neutral    0.618162
true_long       0.215107
dtype: float64

In [ ]:
predicted[predicted['long'] == predicted.iloc[:, :3].max(1)].iloc[:,3:].mean()

true_short      0.156348
true_neutral    0.618360
true_long       0.225292
dtype: float64

In [ ]:
model.summary()